In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D
from keras.layers import Input, Reshape, Dot, Add
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
# from keras.utils.vis_utils import plot_model
import keras
import keras.backend as K
from data_handle import *
from gensim_wrapper import *
from utils import *
import gensim
import random
import numpy as np
import tensorflow as tf
seed_val = 1000
random.seed(seed_val)
np.random.seed(seed_val)
tf.set_random_seed(seed_val)

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def parseVec(file, delimiter, word2int):
    lines = open(file, encoding='utf8').readlines()
    vocab_size, embed_size = [int(s) for s in lines[0].split()]
    vocab_size  = min(len(word2int), vocab_size)
    embeddings = np.zeros((vocab_size + 1, embed_size), dtype=np.float64)
    for i in range(1, vocab_size):
        try:
            line = lines[i][:-1].split(delimiter)
            word = line[0]
            if '*###*' in word or 'unk' in word:
                continue
            if word in word2int:
                wordvec = np.array([np.float64(j) for j in line[1:] if j != ''])
                embeddings[word2int[word]] = wordvec
        except Exception as e:
#             print(lines[i])
            print(e)
    return embeddings

In [3]:
words = read_file(filename='data/news_mini.txt')
words, word2freq = min_count_threshold(words)
vocab, word2int, int2word = build_vocab(words)

In [4]:
xwords = read_file(filename='data/news.txt')
xwords, xword2freq = min_count_threshold(xwords)
xvocab, xword2int, xint2word = build_vocab(xwords)

In [5]:
del word2int['*###*']
del xword2int['*###*']

In [6]:
w2v_mini = parseVec('results/w2v_mini.txt', ' ', word2int)
w2v = parseVec('results/w2v.txt', ' ', word2int=xword2int)

In [76]:
mine = parseVec('results/w2v_high_1_4.txt', ' ', word2int)

In [77]:
mini_utils= Utils(embedding=normalize(w2v_mini), int2word=int2word, word2int=word2int)
mine_utils= Utils(embedding=normalize(mine), int2word=int2word, word2int=word2int)
ref_utils= Utils(embedding=normalize(w2v), int2word=xint2word, word2int=xword2int)

D:\Library\Projects\amharic_word_embedding\data_handle.py:309: RuntimeWarning: invalid value encountered in true_divide
  return array / norms


In [78]:
index = np.random.randint(1000) + 1
index = 229
word = words[index]
print(word, index)
r1 = mine_utils.sorted_sim(word, 20)
r2 = mini_utils.sorted_sim(word, 20)
r3 = ref_utils.sorted_sim(word, 20)
print()

for i in range(len(r1)):
    print("{0}\t\t\t{1}\t\t\t{2}".format(r1[i][0], r2[i][0], r3[i][0]))

መቼ 229

መቼ			መቼ			መቼ
ከምንም			አለብኝ			የት
የኋላ			አዎ			ቆይ
አነጋገር			አቤት			ማን
የገበያ			እሺ			እንዲያው
ይላል			ይመስል			የትና
ብቸኛ			ንጉሡ			እያልኩ
አስገንዝበዋል			ጎልማሳ			እንዴ
ለድርድር			እሱን			ለምንስ
ክስተት			ወይዘሮዋ			ከመቼ
በየቀኑ			ልጁ			ማንስ
የተዘጋጀ			ወንድሜ			በማን
ልመና			ሄዶ			እናንተ
አሁንም			ግራ			እንዴት
ወሳኝ			ተሳፋሪ			ለምን
ተሳፋሪ			የተቀመጠ			አይደል
መቀነስ			ወዲያው			መሰላችሁ
የሚመሩት			ደስ			ማንን
ሁሉን			በሉ			እንጃ
ይኼን			ይኼን			ታዲያ
